In [99]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns 

In [128]:
basic_features = pd.read_csv('basic_features_train.csv') 
time_features = pd.read_csv('time_features_train.csv')
content_features = pd.read_csv('content_features_train.csv')
additional_features = pd.read_csv('additional_features_train.csv')
label_features = pd.read_csv('labels_train.csv')
flow_features = pd.read_csv('flow_features_train.csv')

merged_train = pd.merge(basic_features, time_features, how='left', on='id')
merged_train = pd.merge(merged_train, content_features, how='left', on='id')
merged_train = pd.merge(merged_train, additional_features, how='left', on='id')
merged_train = pd.merge(merged_train, label_features, how='left', on='id').drop("label", axis='columns')
merged_train = pd.merge(merged_train, flow_features, how='left', on='id')

## Data Cleaning

In [129]:
# Mengubah tipe data beberapa atribut sesuai features
merged_train[['sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'spkts', 'dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm']] = merged_train[['sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'spkts', 'dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm']].astype('Int64')

# Mengubah missing value pada is_sm_ips_ports dengan nilai modus
mode_value_1 = merged_train['is_sm_ips_ports'].mode()[0]   
merged_train['is_sm_ips_ports'].fillna(mode_value_1) 

# Mengubah missing value pada is_ftp_login dengan nilai modus
mode_value_2 = merged_train['is_ftp_login'].mode()[0]   
merged_train['is_ftp_login'].fillna(mode_value_2) 

# Mengubah missing value pada atribut dengan tipe data float64 dengan avg (gtw)
float_cols = merged_train.select_dtypes(include=['float64']).columns

for col in float_cols:
    merged_train[col].fillna(merged_train[col].median(),  inplace=True)

# Mengubah missing value pada atribut dengan tipe data int64 dengan avg/median (gtw) 
int_cols = merged_train.select_dtypes(include=['int64']).columns

for col in int_cols:
    merged_train[col].fillna(merged_train[col].median(), inplace=True)

# Menghapus missing value pada atribut dengan tipe data object
merged_train = merged_train.dropna(subset=['state', 'service', 'proto'])
merged_train.isnull().sum()

# Mengubah outlier pada atribut binary is_ftp_login (0 or 1)
merged_train['is_ftp_login'] = np.where(merged_train['is_ftp_login'] > 1, 1, merged_train['is_ftp_login'])

# Melakukan perubahan nama pada atribut tertentu
merged_train['service'] = merged_train['service'].apply(lambda x:"None" if x=="-" else x)
merged_train['attack_cat'] = merged_train['attack_cat'].replace('backdoors','backdoor', regex=True).apply(lambda x: x.strip().lower())

C:\Users\raffa\AppData\Local\Temp\ipykernel_9580\2080813755.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_train[col].fillna(merged_train[col].median(),  inplace=True)
C:\Users\raffa\AppData\Local\Temp\ipykernel_9580\2080813755.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves 